In [1]:
import joblib
import warnings
from scipy import stats
from utils.scaler import *
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell

In [2]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df = pd.read_csv('Database/rainfall_train.csv', index_col=0)
df_test = pd.read_csv('Database/rainfall_test.csv')
train_col = df.columns.tolist()
train_col.remove(train_col[-2])
df_test.columns = train_col
train, self_test_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

# Train General Preprocessing

In [4]:
df_1 = train.copy()
mask = df_1['rainfall_train.class_interval'] == -999
df_1 = df_1[~mask]

delete_list = ['rainfall_train.fc_year', 'rainfall_train.fc_month', 'rainfall_train.fc_day', 'rainfall_train.fc_hour',
               'rainfall_train.ef_year', 'rainfall_train.vv']
df_1 = df_1.drop(columns=delete_list)

# Train 이산형변수 처리

In [5]:
df_2 = df_1.copy()

original_values = list(range(3, 241, 3))
reversed_values = original_values[::-1]
mapping_table = dict(zip(original_values, reversed_values))


def map_value(x):
    if pd.isna(x):
        return x
    return mapping_table.get(x, x)


df_2['rainfall_train.dh'] = df_2['rainfall_train.dh'].apply(map_value)

In [6]:
enc_dict = {}
onehot_df = pd.DataFrame(index=df_2.index)
for i, col in enumerate(['rainfall_train.ef_month', 'rainfall_train.ef_day', 'rainfall_train.ef_hour']):
    enc_dict[i] = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    one_hot_encoded = enc_dict[i].fit_transform(pd.DataFrame(df_2[col]))

    encoded_df = pd.DataFrame(one_hot_encoded, columns=enc_dict[i].get_feature_names_out([col]), index=df_2.index)
    onehot_df = pd.concat([onehot_df, encoded_df], axis=1)
    df_2 = df_2.drop(columns=[col])
df_2 = pd.concat([df_2, onehot_df], axis=1)

# Train 파생변수

In [7]:
df_3 = df_2.copy()

continuous_list = [
    'rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03', 'rainfall_train.v04', 'rainfall_train.v05',
    'rainfall_train.v06', 'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09']

df_3.insert(11, 'cum_prob', df_3[continuous_list].sum(axis=1))
df_3.insert(12, 'Zero_Count', (df_3[continuous_list] == 0).sum(axis=1))

# Train 정규화

In [8]:
df_4 = df_3.copy()

minmax_list = ['rainfall_train.dh', 'Zero_Count']
minmax_scaler = MinMaxScaler()
df_4[minmax_list] = minmax_scaler.fit_transform(df_4[minmax_list])

# Train 연속형변수 분포저장

In [9]:
y = df_4['rainfall_train.class_interval']
X = df_4.drop(columns='rainfall_train.class_interval')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42, shuffle=True)

In [10]:
X_train_norm = X_train.copy()
X_val_norm = X_val.copy()

continuous_list = [
    'rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03', 'rainfall_train.v04', 'rainfall_train.v05',
    'rainfall_train.v06', 'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09', 'cum_prob']

X_train_norm.loc[:, continuous_list], scaler = standard_scale_train(X_train, continuous_list)
X_val_norm.loc[:, continuous_list] = standard_scale_val(X_val, continuous_list, scaler)

# 불균형 해소를 위한 학습파일 저장

In [11]:
total_train = pd.concat([X_train_norm, y_train], axis=1)
total_val = pd.concat([X_val_norm, y_val], axis=1)

total_train = total_train.iloc[:, 1:]
total_val = total_val.iloc[:, 1:]

train_mask = total_train['rainfall_train.class_interval'] == 0
val_mask = total_val['rainfall_train.class_interval'] == 0

In [12]:
# total_train[~train_mask].to_csv('Database/total/total_train.csv')
# total_val[~val_mask].to_csv('Database/total/total_val.csv')

# test General Preprocessing

In [13]:
self_test = True

if self_test:
    df_1 = self_test_df.copy()
if not self_test:
    df_1 = df_test.copy()

mask = df_1['rainfall_train.class_interval'] == -999
df_1 = df_1[~mask]

delete_list = ['rainfall_train.fc_year', 'rainfall_train.fc_month', 'rainfall_train.fc_day', 'rainfall_train.fc_hour',
               'rainfall_train.ef_year', 'rainfall_train.vv']
df_1 = df_1.drop(columns=delete_list)

# test 이산형변수 처리

In [14]:
df_2 = df_1.copy()

original_values = list(range(3, 241, 3))
reversed_values = original_values[::-1]
mapping_table = dict(zip(original_values, reversed_values))


def map_value(x):
    if pd.isna(x):
        return x
    return mapping_table.get(x, x)


df_2['rainfall_train.dh'] = df_2['rainfall_train.dh'].apply(map_value)

In [15]:
onehot_df = pd.DataFrame(index=df_2.index)
for i, col in enumerate(['rainfall_train.ef_month', 'rainfall_train.ef_day', 'rainfall_train.ef_hour']):
    one_hot_encoded = enc_dict[i].transform(pd.DataFrame(df_2[col]))

    encoded_df = pd.DataFrame(one_hot_encoded, columns=enc_dict[i].get_feature_names_out([col]), index=df_2.index)
    onehot_df = pd.concat([onehot_df, encoded_df], axis=1)
    df_2 = df_2.drop(columns=[col])
df_2 = pd.concat([df_2, onehot_df], axis=1)

# test 파생변수

In [16]:
df_3 = df_2.copy()

continuous_list = [
    'rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03', 'rainfall_train.v04', 'rainfall_train.v05',
    'rainfall_train.v06', 'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09']

df_3.insert(11, 'cum_prob', df_3[continuous_list].sum(axis=1))
df_3.insert(12, 'Zero_Count', (df_3[continuous_list] == 0).sum(axis=1))

# test 정규화

In [17]:
df_4 = df_3.copy()

minmax_list = ['rainfall_train.dh', 'Zero_Count']
df_4[minmax_list] = minmax_scaler.transform(df_4[minmax_list])

# test 이산형변수 칼럼통일

In [18]:
if self_test:
    y2 = df_4['rainfall_train.class_interval']
    X2 = df_4.drop(columns='rainfall_train.class_interval')

    # train에만 있고 test에 없는 경우, 해당 column name으로 test에 zero columns 추가.
    X2[list(X_val_norm.columns[X_val_norm.columns.isin(X2) == False])] = 0

    # test에만 있고 train에는 없는 경우, 해당 column name은 제거.
    X2 = X2.drop(columns=list(X2.columns[X2.columns.isin(X_val_norm) == False]))
    X2 = X2[X_val_norm.columns]

if not self_test:
    X2 = df_4

    # train에만 있고 test에 없는 경우, 해당 column name으로 test에 zero columns 추가.
    X2[list(X_val_norm.columns[X_val_norm.columns.isin(X2) == False])] = 0

    # test에만 있고 train에는 없는 경우, 해당 column name은 제거.
    X2 = X2.drop(columns=list(X2.columns[X2.columns.isin(X_val_norm) == False]))
    X2 = X2[X_val_norm.columns]

# test 연속형변수 분포통일

In [19]:
continuous_list = [
    'rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03', 'rainfall_train.v04', 'rainfall_train.v05',
    'rainfall_train.v06', 'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09', 'cum_prob']

X2.loc[:, continuous_list] = standard_scale_val(X2, continuous_list, scaler)

# 뷸균형 해소

In [24]:
import torch
from Module.model import MLP, VAE

backbone = MLP(57, 10)
backbone.load_state_dict(torch.load('Weight/MLP.pth'))
model = VAE(32, backbone, 10000)
model.load_state_dict(torch.load('Weight/VAE.pth'))

<All keys matched successfully>

<All keys matched successfully>

In [25]:
syn_data = model()

In [26]:
syn_df = pd.DataFrame(syn_data.cpu().detach().numpy(),
                      columns=list(X_train_norm.iloc[:, 1:].columns) + ['rainfall_train.class_interval'])

In [28]:
total_train

,rainfall_train.dh,rainfall_train.v01,rainfall_train.v02,rainfall_train.v03,rainfall_train.v04,rainfall_train.v05,rainfall_train.v06,rainfall_train.v07,rainfall_train.v08,rainfall_train.v09,...,rainfall_train.ef_day_31,rainfall_train.ef_hour_0,rainfall_train.ef_hour_3,rainfall_train.ef_hour_6,rainfall_train.ef_hour_9,rainfall_train.ef_hour_12,rainfall_train.ef_hour_15,rainfall_train.ef_hour_18,rainfall_train.ef_hour_21,rainfall_train.class_interval
877611,0.468354,-0.695012,-0.717671,-0.715057,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
633836,0.936709,-1.036971,-0.939973,-0.801748,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
606061,0.455696,-0.934383,-0.865872,-0.758402,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
1214355,0.607595,-1.036971,-0.939973,-0.801748,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
682391,0.949367,-1.036971,-0.939973,-0.801748,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154183,0.063291,0.672826,0.801393,0.932070,1.039805,1.041018,0.515299,0.698355,1.558754,1.679483,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6
95230,0.531646,1.254157,1.283047,1.062106,0.377752,-0.086795,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5
474380,0.886076,-1.036971,-0.939973,-0.801748,-0.691718,-0.588046,-0.446242,-0.325896,-0.191676,-0.125634,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1349211,0.696203,1.527725,1.727651,2.015706,2.160203,1.980863,1.380686,0.356938,-0.191676,-0.125634,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [27]:
syn_df

,rainfall_train.v01,rainfall_train.v02,rainfall_train.v03,rainfall_train.v04,rainfall_train.v05,rainfall_train.v06,rainfall_train.v07,rainfall_train.v08,rainfall_train.v09,cum_prob,...,rainfall_train.ef_hour_0,rainfall_train.ef_hour_3,rainfall_train.ef_hour_6,rainfall_train.ef_hour_9,rainfall_train.ef_hour_12,rainfall_train.ef_hour_15,rainfall_train.ef_hour_18,rainfall_train.ef_hour_21,0,rainfall_train.class_interval
0,0.720797,1.094631,1.006799,0.599714,0.065451,-0.316764,-0.503865,-0.477086,-0.177306,0.013470,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0.388857,1.238083,1.269604,1.291004,1.241317,1.135418,0.803420,0.388910,0.094945,0.072300,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0.931466,2.402789,2.774061,3.505012,4.096755,4.220565,2.299418,0.180811,-0.160676,-0.129963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,0.767014,2.296962,2.643954,3.382947,4.135106,4.811861,4.705983,2.918020,0.708234,0.174206,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,0.423824,0.206676,0.130970,0.026345,-0.055138,-0.116686,-0.186111,-0.234096,-0.185589,-0.094439,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.566948,0.625839,0.506530,0.220510,-0.066878,-0.256943,-0.339826,-0.313767,-0.168709,-0.069567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9996,0.438238,1.492427,1.603440,1.811120,1.948959,2.000411,1.930304,1.946808,1.483439,1.037907,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
9997,0.530467,0.976728,0.883655,0.606953,0.287753,0.029515,-0.125362,-0.120842,-0.215748,-0.239903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
9998,0.421100,-0.222172,-0.273170,-0.304282,-0.309172,-0.299650,-0.258575,-0.285467,-0.206865,-0.096960,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


# 지역별 분리

In [94]:
# X_train_norm_dict = {}
# y_train_norm_dict = {}
# 
# for i, STN in enumerate(df['rainfall_train.stn4contest'].unique().tolist()):
#     train_mask = (X_train_norm['rainfall_train.stn4contest'] == STN)
#     X_train_norm_dict[STN] = X_train_norm[train_mask]
#     y_train_norm_dict[STN] = pd.DataFrame(y_train_norm)[train_mask]
# 
# for i, (key, value) in enumerate(X_train_norm_dict.items()):
#     X_train_norm_dict[key] = value.iloc[:, 1:]
# 
# X_val_norm = X_val_norm.iloc[:, 1:]
# X2 = X2.iloc[:, 1:]

# GMM분리

In [21]:
X_train_norm = X_train_norm.iloc[:, 1:]
X_val_norm = X_val_norm.iloc[:, 1:]
X2 = X2.iloc[:, 1:]

GMM = joblib.load('Database/gmm_model.pkl')
X_train_norm = pd.concat([X_train_norm, pd.DataFrame(GMM.predict(X_train_norm), index=X_train_norm.index)], axis=1)

X_train_norm_dict = {}
y_train_norm_dict = {}

for i, cluster_num in enumerate(X_train_norm[0].unique().tolist()):
    train_mask = (X_train_norm[0] == cluster_num)
    X_train_norm_dict[cluster_num] = X_train_norm[train_mask]
    y_train_norm_dict[cluster_num] = pd.DataFrame(y_train)[train_mask]
    X_train_norm_dict[cluster_num] = X_train_norm_dict[cluster_num].drop(columns=0)

In [22]:
for i, (key, value) in enumerate(X_train_norm_dict.items()):
    print(f'{key} Train set')
    print(value.shape)
    print(y_train_norm_dict[key].shape)

print('Validation set')
print(X_val_norm.shape)
print(y_val.shape)

print('Test set')
print(X2.shape)

if self_test:
    print(y2.shape)

10 Train set
(123072, 57)
(123072, 1)
4 Train set
(97820, 57)
(97820, 1)
1 Train set
(90284, 57)
(90284, 1)
6 Train set
(125411, 57)
(125411, 1)
8 Train set
(150807, 57)
(150807, 1)
9 Train set
(115253, 57)
(115253, 1)
5 Train set
(20187, 57)
(20187, 1)
3 Train set
(54302, 57)
(54302, 1)
7 Train set
(18608, 57)
(18608, 1)
14 Train set
(74820, 57)
(74820, 1)
12 Train set
(93290, 57)
(93290, 1)
0 Train set
(175835, 57)
(175835, 1)
13 Train set
(80389, 57)
(80389, 1)
2 Train set
(5023, 57)
(5023, 1)
11 Train set
(13570, 57)
(13570, 1)
Validation set
(65194, 57)
(65194,)
Test set
(144897, 57)
(144897,)


# 전처리 파일저장

In [23]:
for i, (key, value) in enumerate(X_train_norm_dict.items()):
    value.to_csv(f'Database/train/GMM{key}_X_train_norm.csv')
    y_train_norm_dict[key].to_csv(f'Database/train/GMM{key}_y_train.csv')
    print(f'GMM{key} Train set saved!')

X_val_norm.to_csv(f'Database/val/X_val_norm.csv')
y_val.to_csv(f'Database/val/y_val.csv')
print('Validation set saved!')

if self_test:
    X2.to_csv(f'Database/test/X_self_test_norm.csv')
    y2.to_csv(f'Database/test/y_self_test.csv')

if not self_test:
    X2.to_csv(f'Database/test/X_real_test_norm.csv')
print('Test set saved!')

GMM10 Train set saved!
GMM4 Train set saved!
GMM1 Train set saved!
GMM6 Train set saved!
GMM8 Train set saved!
GMM9 Train set saved!
GMM5 Train set saved!
GMM3 Train set saved!
GMM7 Train set saved!
GMM14 Train set saved!
GMM12 Train set saved!
GMM0 Train set saved!
GMM13 Train set saved!
GMM2 Train set saved!
GMM11 Train set saved!
Validation set saved!
Test set saved!
